# bonus_unit 1 内容

这一节的内容主要是从零微调一个带有tool的调用

在我个人的认知里，抄一遍是没有用的，得自己做一遍才有用

## 先挑选一个模型

我们需要一个参数较少，我们可以训的起，此外，需要一个暂时没有工具能力的llm

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
checkpoint = "Qwen/Qwen2.5-0.5B"
llm = AutoModelForCausalLM.from_pretrained(checkpoint, torch_dtype=torch.bfloat16).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [3]:
tools = [
    {
        "type": "function",
        "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
            "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA"
            },
            "unit": {
                "type": "string",
                "enum": ["celsius", "fahrenheit"]
            }
            },
            "required": ["location"]
        }
        }
    }
]
messages = [
    {
        "role": "user",
        "content": "What'\''s the weather like in Boston today?"
    }
]

inputs = tokenizer.apply_chat_template(messages, tools=tools, add_generation_prompt=True, tokenize=False)

In [4]:
inputs

'<|im_start|>system\nYou are a helpful assistant.\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"type": "function", "function": {"name": "get_current_weather", "description": "Get the current weather in a given location", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": "The city and state, e.g. San Francisco, CA"}, "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]}}, "required": ["location"]}}}\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{"name": <function-name>, "arguments": <args-json-object>}\n</tool_call><|im_end|>\n<|im_start|>user\nWhat\'\'\'s the weather like in Boston today?<|im_end|>\n<|im_start|>assistant\n'

In [5]:
inputs = tokenizer([inputs], return_tensors="pt").to(llm.device)
response = llm.generate(**inputs, max_new_tokens=512)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [6]:
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, response)
]

response_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [7]:
response_text

'{"name": "get_current_weather", "arguments": {"location": "Boston", "unit": "celsius"}}\neniable\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant\nitant

mua的  这个问题至今不理解， 这个模型讲道理应该只做了pretrain  不知道为啥能做chat

但是不管了，反正也能说明对工具不行

In [8]:
from datasets import load_dataset

In [9]:
raw_dataset = "Jofthomas/hermes-function-calling-thinking-V1"
raw_dataset = load_dataset(raw_dataset)

In [10]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['conversations'],
        num_rows: 3570
    })
})

In [11]:
raw_dataset["train"][0]

{'conversations': [{'content': "You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'get_stock_price', 'description': 'Get the current stock price of a company', 'parameters': {'type': 'object', 'properties': {'company': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company']}}}, {'type': 'function', 'function': {'name': 'get_movie_details', 'description': 'Get details about a movie', 'parameters': {'type': 'object', 'properties': {'title': {'type': 'string', 'description': 'The title of the movie'}}, 'required': ['title']}}}] </tools>Use the following pydantic model json schema for each tool call you will make: {'title': 'FunctionCall', 'type': 'object', 'properties': {'arguments': {'title'

In [12]:
tokenizer.encode("<think>")

[13708, 766, 29]

In [13]:
# 需要增加一些token给我们的模型
from enum import Enum

class ChatmlSpecialTokens(str, Enum):
    tools = "<tools>"
    eotools = "</tools>"
    think = "<think>"
    eothink = "</think>"
    tool_call="<tool_call>"
    eotool_call="</tool_call>"
    tool_response="<tool_reponse>"
    eotool_response="</tool_reponse>"
    pad_token = "<pad>"
    eos_token = "<eos>"
    @classmethod
    def list(cls):
        return [c.value for c in cls]
    
# 这里直接复制了教程里提供的Token 事实上这里有些token是不需要的，对于我们的Qwen来说 但是图省事
tokenizer = AutoTokenizer.from_pretrained(
    checkpoint,
    additional_special_tokens=ChatmlSpecialTokens.list()
)

这里数据查看，更说明了一点，他这个数据里带think的，而我们模型不带，说明可以训

In [14]:
# 因为完整数据是3570 按3500:70分割
raw_dataset = raw_dataset["train"].train_test_split(test_size=70, seed=42)

In [15]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['conversations'],
        num_rows: 3500
    })
    test: Dataset({
        features: ['conversations'],
        num_rows: 70
    })
})

In [16]:
def process_function(examples):
    # 这里跟教程不一样 因为我们不使用trl包  而是使用Trainer 需要是token_id的形式
    text_list = tokenizer.apply_chat_template(examples["conversations"], add_generation_prompt=True, tokenize=False)
    return tokenizer(text_list)


dataset = raw_dataset.map(process_function, batched=True, remove_columns=["conversations"])
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 3500
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 70
    })
})

In [17]:
dataset["train"][0]

{'input_ids': [151644,
  8948,
  198,
  2610,
  525,
  264,
  729,
  8098,
  15235,
  1614,
  13,
  1446,
  525,
  3897,
  448,
  729,
  32628,
  2878,
  220,
  151665,
  151666,
  11874,
  9492,
  38437,
  1231,
  1618,
  825,
  476,
  803,
  5746,
  311,
  7789,
  448,
  279,
  1196,
  3239,
  13,
  4320,
  944,
  1281,
  31846,
  911,
  1128,
  2750,
  311,
  19633,
  1119,
  5746,
  90073,
  525,
  279,
  2500,
  7375,
  25,
  151665,
  61108,
  1313,
  1210,
  364,
  1688,
  516,
  364,
  1688,
  1210,
  5360,
  606,
  1210,
  364,
  1836,
  32231,
  33984,
  516,
  364,
  4684,
  1210,
  364,
  5890,
  369,
  15556,
  3118,
  389,
  3728,
  323,
  35005,
  516,
  364,
  13786,
  1210,
  5360,
  1313,
  1210,
  364,
  1700,
  516,
  364,
  13193,
  1210,
  5360,
  2527,
  1210,
  5360,
  1313,
  1210,
  364,
  917,
  516,
  364,
  4684,
  1210,
  364,
  785,
  3728,
  311,
  2711,
  369,
  15556,
  24731,
  364,
  66,
  55140,
  1210,
  5360,
  1313,
  1210,
  364,
  917,
  516,
 

In [18]:
llm.resize_token_embeddings(len(tokenizer))  # 会修改model.vocab_size

Embedding(151673, 896)

In [19]:
from peft import LoraConfig, TaskType, get_peft_model

In [20]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
)

llm_lora = get_peft_model(llm, lora_config)

In [21]:
llm_lora.print_trainable_parameters()

trainable params: 540,672 || all params: 494,337,792 || trainable%: 0.1094


In [22]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

In [23]:
args = TrainingArguments(
    "output",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    learning_rate=5e-5,
    report_to="wandb",
    save_strategy="epoch",
    logging_steps=5,
    bf16=True,
    gradient_accumulation_steps=5,
    eval_strategy="epoch",
    do_eval=True
)
trainer = Trainer(
    llm_lora, 
    args, 
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"]
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [24]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: yizhen-ciao (yizhen-ciao-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.660400,0.661976
2,0.498200,0.528925
3,0.503200,0.513666


d:\software\anaconda\envs\huggingface_agent_course\Lib\site-packages\peft\utils\save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
d:\software\anaconda\envs\huggingface_agent_course\Lib\site-packages\peft\utils\save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
d:\software\anaconda\envs\huggingface_agent_course\Lib\site-packages\peft\utils\save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
d:\software\anaconda\envs\huggingface_agent_course\Lib\site-packages\peft\utils\save_and_load.py:260: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


TrainOutput(global_step=525, training_loss=0.832591903323219, metrics={'train_runtime': 879.1027, 'train_samples_per_second': 11.944, 'train_steps_per_second': 0.597, 'total_flos': 1.1381457960677376e+16, 'train_loss': 0.832591903323219, 'epoch': 3.0})

## 训完测效果

In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraModel, PeftConfig
from datasets import load_dataset

In [2]:
checkpoint = "Qwen/Qwen2.5-0.5B"
llm = AutoModelForCausalLM.from_pretrained(checkpoint, torch_dtype=torch.bfloat16).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [3]:
# 需要增加一些token给我们的模型
from enum import Enum

class ChatmlSpecialTokens(str, Enum):
    tools = "<tools>"
    eotools = "</tools>"
    think = "<think>"
    eothink = "</think>"
    tool_call="<tool_call>"
    eotool_call="</tool_call>"
    tool_response="<tool_reponse>"
    eotool_response="</tool_reponse>"
    pad_token = "<pad>"
    eos_token = "<eos>"
    @classmethod
    def list(cls):
        return [c.value for c in cls]
    
# 这里直接复制了教程里提供的Token 事实上这里有些token是不需要的，对于我们的Qwen来说 但是图省事
tokenizer = AutoTokenizer.from_pretrained(
    checkpoint,
    additional_special_tokens=ChatmlSpecialTokens.list()
)

In [4]:
lora_config = PeftConfig.from_pretrained("output\checkpoint-525")
llm_lora = LoraModel(llm, config=lora_config, adapter_name="tools")

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_7004\2513183782.py:1: SyntaxWarning: invalid escape sequence '\c'
  lora_config = PeftConfig.from_pretrained("output\checkpoint-525")


In [5]:
tools = [
    {
        "type": "function",
        "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
            "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA"
            },
            "unit": {
                "type": "string",
                "enum": ["celsius", "fahrenheit"]
            }
            },
            "required": ["location"]
        }
        }
    }
]
messages = [
    {
        "role": "user",
        "content": "What'\''s the weather like in Boston today?"
    }
]

inputs = tokenizer.apply_chat_template(messages, tools=tools, add_generation_prompt=True, tokenize=False)

In [6]:
inputs

'<|im_start|>system\nYou are a helpful assistant.\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"type": "function", "function": {"name": "get_current_weather", "description": "Get the current weather in a given location", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": "The city and state, e.g. San Francisco, CA"}, "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]}}, "required": ["location"]}}}\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{"name": <function-name>, "arguments": <args-json-object>}\n</tool_call><|im_end|>\n<|im_start|>user\nWhat\'\'\'s the weather like in Boston today?<|im_end|>\n<|im_start|>assistant\n'

In [7]:
inputs = tokenizer([inputs], return_tensors="pt").to(llm_lora.device)

In [8]:
response = llm_lora.generate(**inputs, max_new_tokens=512)
response

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,    382,
              2,  13852,    271,   2610,   1231,   1618,    825,    476,    803,
           5746,    311,   7789,    448,    279,   1196,   3239,    382,   2610,
            525,   3897,    448,    729,  32628,   2878,    220, 151665, 151666,
          11874,   9492,    510, 151665,    198,   4913,   1313,    788,    330,
           1688,    497,    330,   1688,    788,   5212,    606,    788,    330,
            455,  11080,  69364,    497,    330,   4684,    788,    330,   1949,
            279,   1482,   9104,    304,    264,   2661,   3728,    497,    330,
          13786,    788,   5212,   1313,    788,    330,   1700,    497,    330,
          13193,    788,   5212,   2527,    788,   5212,   1313,    788,    330,
            917,    497,    330,   4684,    788,    330,    785,   3283,    323,
           1584,     11,    384,   1302,     13,   5836,  12879,     11,   9183,
          14345,    330,   3

In [9]:
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, response)
]

response_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [11]:
print(response_text)

{"name": "get_current_weather", "arguments": {"location": "Boston", "unit": "celsius"}}
ponde
What'''s the weather like in Boston today?ponde
ponde
What'''s the weather like in Boston today?ponde
ponde
What'''s the weather like in Boston today?ponde
ponde
What'''s the weather like in Boston today?ponde
ponde
What'''s the weather like in Boston today?ponde
ponde
What'''s the weather like in Boston today?ponde
ponde
What'''s the weather like in Boston today?ponde
ponde
What'''s the weather like in Boston today?ponde
ponde
What'''s the weather like in Boston today?ponde
ponde
What'''s the weather like in Boston today?ponde
ponde
What'''s the weather like in Boston today?ponde
ponde
What'''s the weather like in Boston today?ponde
ponde
What'''s the weather like in Boston today?ponde
ponde
What'''s the weather like in Boston today?ponde
ponde
What'''s the weather like in Boston today?ponde
ponde
What'''s the weather like in Boston today?ponde
ponde
What'''s the weather like in Boston today?

> 这个要重新训， 有问题

应该是需要训练数据需要重新解析一下，tools 以及结束符号有问题，待做